In [16]:
import popexposure as pe
import pandas as pd
import geopandas as gpd
import numpy as np
import math
from IPython.display import Markdown as md

In [17]:
wfbz_all = gpd.read_file('../data/raw/wfbz_disasters_2000-2025.geojson')
wfbz_all = wfbz_all[wfbz_all['wildfire_community_intersect'] == True]

wfbz = wfbz_all[~wfbz_all.geometry.is_empty].copy()
wfbz['ID_hazard'] = wfbz['wildfire_id']
wfbz['buffer_dist_10km'] = 10_000
wfbz = {group_name: group_data for group_name, group_data in wfbz.groupby('wildfire_year')}
pop = {
  2000 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2000/GHS_POP_E2000_GLOBE_R2023A_54009_100_V1_0.tif'),
  2005 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2005/GHS_POP_E2005_GLOBE_R2023A_54009_100_V1_0.tif'),
  2010 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2010/GHS_POP_E2010_GLOBE_R2023A_54009_100_V1_0.tif'),
  2015 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2015/GHS_POP_E2015_GLOBE_R2023A_54009_100_V1_0.tif'),
  2020 : pe.PopEstimator(pop_data='../data/raw/ghs_pop/2020/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif'),
}


In [18]:
pop_impacted = dict()
for yr in range(2000,2026):
  print(yr)
  pop_impacted[yr] = pop[min(math.floor(yr/5)*5, 2020)].est_exposed_pop(
    hazard_data=wfbz[yr],
    hazard_specific=False
  )

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025


In [19]:
pop_impacted_all = pd.concat(pop_impacted.values(), ignore_index=True)
pop_impacted_all['year'] = range(2000,2026)

In [30]:
md(f"""Studies document increasing wildfire smoke exposure in the United States, but no
research has characterized the locations of wildfires that impact human populations (i.e.,
wildfire burn zone disasters [WFBZ disasters]). Here, we leverage a novel dataset of WFBZ
disasters by harmonizing six existing national or California-specific datasets to
characterize spatiotemporal trends in WFBZ disaster occurrence, size, and severity,
populations impacted, intersection with the wildland-urban interface, and FEMA disaster
declarations. Between 2000-2025, we identified {len(wfbz_all):,.0f} WFBZ disasters in the United States.
These WFBZ disasters burned a cumulative {(wfbz_all.to_crs(3857).dissolve().area/1000/1000).iloc[0]:,.0f} km<sup>2</sup>, with the burned area exceeding
20,000km<sup>2</sup> in Texas, Idaho, Oregon, and California. On average, {pop_impacted_all.exposed_10km.mean():,.0f} people lived within 10km
km of a WFBZ annually, with a {(100*((pop_impacted_all.exposed_10km / pop_impacted_all.exposed_10km.shift(1)).mean() - 1)):,.0f}% average increase in count annually during the study
period. California and Hawaii WFBZ disasters accounted for {100*wfbz_all[wfbz_all['wildfire_states'].isin(['CA', 'HI'])]['wildfire_max_civil_fatalities'].sum() / wfbz_all['wildfire_max_civil_fatalities'].sum():,.0f}% (n={wfbz_all[wfbz_all['wildfire_states'].isin(['CA', 'HI'])]['wildfire_max_civil_fatalities'].sum()}) of civilian
fatalities and California alone accounted for the majority of structures destroyed ({(100*(wfbz_all[wfbz_all['wildfire_states']=='CA'].wildfire_struct_destroyed.sum()/wfbz_all.wildfire_struct_destroyed.sum())):,.0f}%,
n={(wfbz_all[wfbz_all['wildfire_states']=='CA'].wildfire_struct_destroyed.sum()):,.0f}). We find that certain states were more likely to receive a FEMA disaster declaration
for WFBZ disasters. For example, Hawaii and New Hampshire received FEMA disaster
declarations for >85% of identified WFBZ disasters, while California, Montana, and Texas
received one for 20-30% of WFBZ disasters. Understanding trends in WFBZ locations,
severity, and populations exposed can aid in disaster preparedness and recovery efforts.""")

Studies document increasing wildfire smoke exposure in the United States, but no
research has characterized the locations of wildfires that impact human populations (i.e.,
wildfire burn zone disasters [WFBZ disasters]). Here, we leverage a novel dataset of WFBZ
disasters by harmonizing six existing national or California-specific datasets to
characterize spatiotemporal trends in WFBZ disaster occurrence, size, and severity,
populations impacted, intersection with the wildland-urban interface, and FEMA disaster
declarations. Between 2000-2025, we identified 6,097 WFBZ disasters in the United States.
These WFBZ disasters burned a cumulative 394,614 km<sup>2</sup>, with the burned area exceeding
20,000km<sup>2</sup> in Texas, Idaho, Oregon, and California. On average, 4,907,711 people lived within 10km
km of a WFBZ annually, with a 18% average increase in count annually during the study
period. California and Hawaii WFBZ disasters accounted for 61% (n=324.0) of civilian
fatalities and California alone accounted for the majority of structures destroyed (70%,
n=85,752). We find that certain states were more likely to receive a FEMA disaster declaration
for WFBZ disasters. For example, Hawaii and New Hampshire received FEMA disaster
declarations for >85% of identified WFBZ disasters, while California, Montana, and Texas
received one for 20-30% of WFBZ disasters. Understanding trends in WFBZ locations,
severity, and populations exposed can aid in disaster preparedness and recovery efforts.

In [28]:
wfbz_all[wfbz_all['wildfire_states'].isin(['CA', 'HI'])]['wildfire_max_civil_fatalities'].sum() / wfbz_all['wildfire_max_civil_fatalities'].sum()
#wfbz_all[wfbz_all['wildfire_states'].isin(['CA', 'HI'])]['wildfire_max_civil_fatalities'].sum()


np.float64(0.6136363636363636)